In [1]:
from pathlib import Path
import json

import geopandas as gpd
import polars as pl

In [2]:
evoland = gpd.read_file(
    Path(
        "//digs110/FER/EvoLand/WP2_6_CFM/Referenzdaten/Database/PointMultitemp/evoland_ref_points_mt.shp"
    )
)

In [5]:
evoland_points = evoland.drop_duplicates("id").drop_duplicates("geometry")
evoland_points["data_loc"] = "./tiffs/" + evoland_points["id"].astype(str)

In [6]:
(
    evoland_points[["id", "data_loc", "geometry"]]
    # .query("id > 915")
    .to_file("../data/evo_tsbrowser.gpkg")
)

In [27]:
evoland_label_mapping = {
    "gesund": "0",
    "brand": "8",
    "raeumung": "a",  # needs to be handled more specifically, to refine abiotic, biotic
    "entnahme": "5",  # clear cut
    "kaefer": "1",
    "sturm": "7",
    "erholung": "3",  # erholung only happens after schwammspinner and does not have an upper bound -> so canopy closing
    "teilraeumung": "a",
    "sturm-teil": "7",
    "durchforstung": "6",
    "kaefer-teil": "1",
    "fraesen": "c",
    "unknown": "e",
    "trockenstress": "9",
    "schwammspinner": "2",
    "verjuengung": "6",  # thinning
    "sturm_snow": "7",
    "sturm-snow": "7",
    "unklar": "e",
    "befallsmax": "1",
}

In [42]:
evoland.columns

Index(['id', 'poly_id', 'batch', 'tile', 'site', 'id_str', 'flag', 'state',
       'datetime', 'oid', 'blue', 'green', 'red', 'nir', 're1', 're2', 're3',
       'B8A', 'swir1', 'swir2', 'comment1', 'comment2', 'comment3', 'obs_qual',
       'fty', 'month', 'dslf', 'dtnf', 'obsQualSt1', 'TCW', 'geometry'],
      dtype='object')

In [ ]:
small_evo = pl.from_pandas(
    evoland[["id", "id_str", "datetime", "state", "comment1", "comment2", "comment3"]]
)
small_evo

id,id_str,datetime,state,comment1,comment2,comment3
i32,str,str,str,str,str,str
0,"""0000""","""20190104T111106""",null,"""leichte Durchforstung 2021""",null,null
0,"""0000""","""20190109T111103""",null,"""leichte Durchforstung 2021""",null,null
0,"""0000""","""20190114T111107""",null,"""leichte Durchforstung 2021""",null,null
0,"""0000""","""20190124T111108""",null,"""leichte Durchforstung 2021""",null,null
0,"""0000""","""20190129T111105""",null,"""leichte Durchforstung 2021""",null,null
…,…,…,…,…,…,…
1267,"""1267""","""20210905T101633""","""kaefer-teil""","""22""",null,null
1267,"""1267""","""20210910T101637""","""kaefer-teil""","""22""",null,null
1267,"""1267""","""20211010T101642""","""kaefer-teil""","""22""",null,null


In [ ]:
segments = ["0", "1", "2", "3", "4", "b", "9"]

cleaned_segments = (
    small_evo.select(
        pl.col.id,
        pl.col.id_str,
        pl.col.datetime.str.strptime(pl.Datetime, "%Y%m%dT%H%M%S"),
        pl.col.state.replace_strict(evoland_label_mapping),
        comment=pl.format(
            "{} {} {}",
            pl.col.comment1.fill_null(""),
            pl.col.comment2.fill_null(""),
            pl.col.comment3.fill_null(""),
        ).str.strip_chars(" "),
    )
    .group_by("id", "state")
    .agg(
        pl.col.id_str.first(),
        pl.col.datetime.min().alias("start"),
        pl.col.datetime.max().alias("end"),
        pl.col.comment.first(),
    )
)
cleaned_segments

id,state,id_str,start,end,comment
i32,str,str,datetime[μs],datetime[μs],str
991,"""1""","""0991""",2019-08-08 10:36:34,2020-02-19 10:36:26,"""22 07-10-2019"""
164,"""0""","""0164""",2019-01-09 11:11:03,2021-10-01 00:00:00,"""Waldbrand_2021_09_08"""
156,"""0""","""0156""",2019-01-09 11:11:03,2021-10-01 00:00:00,"""Waldbrand_2021_09_08 Flußbett"""
916,"""a""","""0916""",2020-04-04 10:36:26,2021-09-26 10:36:36,"""22 07-10-2019"""
509,"""5""","""0509""",2017-05-03 10:40:24,2019-11-04 10:43:39,"""Kahlschlag vor 4/2018 0"""
…,…,…,…,…,…
1228,"""0""","""1228""",2015-08-13 10:16:17,2018-09-19 10:20:18,"""1 23-09-2018"""
1148,"""0""","""1148""",2015-08-06 10:20:12,2019-09-22 10:36:29,""""""
938,"""1""","""0938""",2019-08-23 10:36:35,2020-01-15 10:36:24,"""22 07-10-2019"""


In [ ]:
for id, group in cleaned_segments.group_by("id"):
    comment = group["comment"][0]
    flags = {}
    for row in group.sort("start").iter_rows(named=True):
        state = row["state"]
        if state is None:
            continue
        flags[row["start"].strftime("%Y-%m-%d 00:00:00.000000")] = state
        if state in segments:
            flags[row["end"].strftime("%Y-%m-%d 00:00:00.000000")] = state
    flag_json = {
        "flags": flags,
        "confidence": None,
        "comment": comment,
        "interpreter": "evo",
    }
    with open(f"../data/evoland_flags/flags_{id[0]}.json", "w") as f:
        json.dump(flag_json, f, indent=4)

In [93]:
with open("../data/evoland_flags/flags_2.json") as f:
    flag = json.load(f)

previous_confidence = None
confidence_input = "".strip().lower()
not confidence_input and previous_confidence is not None

False

In [94]:
small_evo.filter(id=39)

id,id_str,datetime,state,comment1,comment2,comment3
i32,str,str,str,str,str,str
39,"""0039""","""20190104T111106""",null,"""Waldbrand_2021_09_08""",null,null
39,"""0039""","""20190109T111103""",null,"""Waldbrand_2021_09_08""",null,null
39,"""0039""","""20190114T111107""",null,"""Waldbrand_2021_09_08""",null,null
39,"""0039""","""20190124T111108""",null,"""Waldbrand_2021_09_08""",null,null
39,"""0039""","""20190129T111105""",null,"""Waldbrand_2021_09_08""",null,null
…,…,…,…,…,…,…
39,"""0039""","""20221201T000000""",null,"""Waldbrand_2021_09_08""",null,null
39,"""0039""","""20221215T000000""",null,"""Waldbrand_2021_09_08""",null,null
39,"""0039""","""20221219T111107""",null,"""Waldbrand_2021_09_08""",null,null
